In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

2024-10-29 19:16:39.795416: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 19:16:39.966260: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 19:16:40.013730: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 19:16:40.325196: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-29 19:16:42.901786: W tensorflow/compiler/tf2

In [2]:



def parse_file(content):
    x_data=[]
    y_data=[]
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    for line in lines:
        # Remove any whitespace and split by '],['
        arrays = line.replace(' ', '').strip('[]').split('],[')
        
        if len(arrays) == 2:
            try:
                # Parse integers for X
                x_array = [int(x) for x in arrays[0].split(',')]
                # Parse floats for Y
                y_array = [float(x) for x in arrays[1].split(',')]
                
                # Only append if arrays have correct dimensions
                if len(x_array) == 24 and len(y_array) == 12:
                    x_data.append(x_array)
                    y_data.append(y_array)
            except (ValueError, IndexError):
                print(f"Skipping malformed line: {line}")
                continue
    return x_data, y_data

def load_data_from_folder(folder):
    all_x = []
    all_y = []
    data_files = [file for root, dirs, files in os.walk(folder) for file in files if file.endswith('.csv')]
    for data_file in data_files:
        with open(os.path.join(folder,data_file), 'r') as file:
            content = file.read()
            x,y = parse_file(content)
            all_x += x
            all_y += y
    X = np.array(all_x, dtype=np.int32)
    Y = np.array(all_y, dtype= np.float32)
    return X,Y

In [3]:
def create_ffnn_model():
    """
    Creates a Feed-Forward Neural Network model for predicting 
    24 float values from 12 input values
    """
    model = Sequential([
        InputLayer(shape=(12,)),
        # Input layer
        Dense(24, activation='relu'),
        BatchNormalization(),
        
        # Hidden layers
        Dense(48, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(48, activation='relu'),
        BatchNormalization(),
        Dropout(0.2),
        
        # Output layer
        Dense(24, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

def train_model(X_train, y_train, epochs=100, batch_size=32):
    """
    Train the model with the provided data
    
    Parameters:
    X_train: numpy array of shape (n_samples, 12)
    y_train: numpy array of shape (n_samples, 24)
    """
    model = create_ffnn_model()
    
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )
    
    history = model.fit(
        X_train, 
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    return model, history

def evaluate_model(model, X_test, y_test):
    """
    Evaluate the model performance
    """
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    
    # Make some predictions
    predictions = model.predict(X_test[:5])  # First 5 samples
    print("\nSample Predictions vs Actual Values:")
    for i in range(5):
        print(f"\nSample {i+1}:")
        print("Prediction:", predictions[i].round(3))
        print("Actual:", y_test[i])

In [4]:
data_folder = "pcpvectors"
# y,X for maintaining convention from this point onwards
y,X = load_data_from_folder(data_folder) 
print(f"\nLoaded {len(X)} samples from CSV files")
print(f"Input shape: {X.shape}")
print(f"Output shape: {y.shape}")


Loaded 5692 samples from CSV files
Input shape: (5692, 12)
Output shape: (5692, 24)


In [5]:
batch_size = min(32, int(np.sqrt(len(X))))
print(batch_size)
print("\nTraining model...")
train_ratio = 0.8
train_size = int(len(X) * train_ratio)

# Shuffle and split
indices = np.random.permutation(len(X))
train_indices, test_indices = indices[:train_size], indices[train_size:]

X_train, X_test = X[train_indices], X[test_indices]
y_train, y_test = y[train_indices], y[test_indices]
model, history = train_model(X_train,y_train, epochs=100, batch_size=batch_size)

32

Training model...


I0000 00:00:1730209605.924587     860 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730209606.305230     860 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730209606.305352     860 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730209606.310670     860 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1730209606.310750     860 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Epoch 1/100


I0000 00:00:1730209611.673713    2822 service.cc:146] XLA service 0x7f9ea0006be0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730209611.673777    2822 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-10-29 19:16:51.764559: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-29 19:16:52.255360: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 44/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0444 - loss: 0.8293

I0000 00:00:1730209614.758208    2822 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


114/114 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.1000 - loss: 0.7754 - val_accuracy: 0.1284 - val_loss: 0.5506
Epoch 2/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3883 - loss: 0.4800 - val_accuracy: 0.3381 - val_loss: 0.2469
Epoch 3/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4737 - loss: 0.2040 - val_accuracy: 0.5598 - val_loss: 0.1381
Epoch 4/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5308 - loss: 0.1318 - val_accuracy: 0.5862 - val_loss: 0.1031
Epoch 5/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5724 - loss: 0.1096 - val_accuracy: 0.6147 - val_loss: 0.0849
Epoch 6/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6180 - loss: 0.0985 - val_accuracy: 0.6454 - val_loss: 0.0754
Epoch 7/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6385 - loss: 0.0903 - val_accuracy: 0.6850 - val_loss: 0.0688
Epoch 8/100
114/114 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6655 - loss: 0.0857 - val_accuracy: 0.72

In [6]:
print("\nEvaluating model...")
evaluate_model(model, X_test,y_test)


Evaluating model...

Test Loss: 0.0320
Test Accuracy: 0.8709
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 619ms/step

Sample Predictions vs Actual Values:

Sample 1:
Prediction: [0.    0.    0.    0.    0.    0.    0.003 0.987 0.    0.    0.    0.
 0.006 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.   ]
Actual: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 2:
Prediction: [0.002 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.959 0.021
 0.001 0.    0.    0.002 0.    0.    0.    0.002 0.001 0.    0.    0.   ]
Actual: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 3:
Prediction: [0.    0.    0.979 0.    0.    0.    0.    0.    0.    0.    0.    0.003
 0.    0.    0.    0.    0.001 0.    0.    0.    0.    0.002 0.    0.   ]
Actual: [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]

Sample 4:
Prediction: [0.    0.    0.    0.    0.    0.    0.    0.    0.002 0.001 0.    0.
 0.    0.001 0.    0.    0.    0.    0.131 0.921 0.    0.    0.    0.   ]
Actual: [0 0 

In [7]:
%%script false --no-raise-error
import matplotlib.pyplot as plt

def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Training MAE')
    plt.plot(history.history['val_mae'], label='Validation MAE')
    plt.title('Model MAE')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# After training:
plot_training_history(history) 

In [8]:
model.save('model.keras')
